In [1]:
typealias Visitor<T> = (T) -> Unit

class BinaryNode<T : Any>(var value: T) {
    var leftChild: BinaryNode<T>? = null
    var rightChild: BinaryNode<T>? = null

    override fun toString(): String = diagram(this)
    
    private fun diagram(
        node: BinaryNode<T>?,
        top: String = "",
        root: String = "",
        bottom: String = ""
    ): String = node?.let { 
        if (node.leftChild == null && node.rightChild == null) {
            "$root${node.value}\n"
        } else {
            diagram(node.rightChild, "$top ", "$top┌──", "$top| ") + 
            root + "${node.value}\n" + diagram(node.leftChild, "$bottom| ", "$bottom└──", "$bottom ")
        }
     } ?: "${root}null\n"
}

fun createTree(): BinaryNode<Int> {
    val zero = BinaryNode(0)
    val one = BinaryNode(1)
    val five = BinaryNode(5)
    val seven = BinaryNode(7)
    val eight = BinaryNode(8)
    val nine = BinaryNode(9)

    seven.leftChild = one
    one.leftChild = zero
    one.rightChild = five
    seven.rightChild = nine
    nine.leftChild = eight
    return seven
}

val tree = createTree()
println(tree)


 ┌──null
┌──9
| └──8
7
| ┌──5
└──1
 └──0


# Traversal algorithms

## In-order traversal

- Start from the root node
- If the current node has a left child, recursively visit this child first.
- Then visit the node itself.
- If the current node has a right child, recursively visit this child.

![In-order traversal](in-order_traversal.png)

0,1,5,7,8,9

In [2]:
fun <T: Any> BinaryNode<T>.traverseInOrder(visit: Visitor<T>) {
    leftChild?.traverseInOrder(visit)
    visit(value)
    rightChild?.traverseInOrder(visit)
}

You first traverse to the left-most node before visiting the value. You then traverse to the right-most node.

In [3]:
tree.traverseInOrder { println(it) }

0
1
5
7
8
9


## Pre-order traversal

- Visits the current node first.
- Recursively visits the left end right child.

![Pre-order traversal](pre-order_traversal.png)

In [4]:
fun <T: Any> BinaryNode<T>.traversePreOrder(visit: Visitor<T>) {
    visit(value)
    leftChild?.traversePreOrder(visit)
    rightChild?.traversePreOrder(visit)
}

tree.traversePreOrder { println(it) }

7
1
0
5
9
8


## Post-order traversal

Post-order traversal always visits the nodes of a binary tree in the following order:
- Recursively visits the left and right child.
- Only visits the current node after the left and right child have been visited recursively.

![Post-order traversal](post-order_traversal.png)

In other words, given any node, you'll visit its children before visiting itself. The root node is always visited last.

In [5]:
fun <T: Any> BinaryNode<T>.traversePostOrder(visit: Visitor<T>) {
    leftChild?.traversePostOrder(visit)
    rightChild?.traversePostOrder(visit)
    visit(value)
}

tree.traversePostOrder { println(it) }

0
5
1
8
9
7


Each one of these traversal algorithms has both a ime and space complexity of **O(n)**.

# Challenges

## Challenge 1: The height of the tree

Given a binary tree, find the height of the tree. The height of the binary tree is determined by the distance between the root and the furthest leaf.

In [6]:
fun <T : Any> BinaryNode<T>.getHeight(
    myLevel: Int = 0
): Int {
    val leftDepth = leftChild?.getHeight(myLevel + 1) ?: myLevel
    val rightDepth = rightChild?.getHeight(myLevel + 1) ?: myLevel
    return if (leftDepth > rightDepth) leftDepth else rightDepth
}

println("Depth: ${tree.getHeight()}")

Depth: 2


In [7]:
fun createTree2(): BinaryNode<Int> {
    val zero = BinaryNode(0)
    val one = BinaryNode(1)
    val two = BinaryNode(2)
    val three = BinaryNode(3)
    val five = BinaryNode(5)
    val seven = BinaryNode(7)
    val eight = BinaryNode(8)
    val nine = BinaryNode(9)

    seven.leftChild = one
    one.leftChild = zero
    one.rightChild = five
    seven.rightChild = nine
    nine.leftChild = eight
    eight.leftChild = two
    two.rightChild = three
    return seven
}

val tree2 = createTree2()
println(tree2)
println("T2 Deepth: ${tree2.getHeight()}")


 ┌──null
┌──9
| | ┌──null
| └──8
|  | ┌──3
|  └──2
|   └──null
7
| ┌──5
└──1
 └──0

T2 Deepth: 4


In [8]:
val oneNodeTree = BinaryNode(0)
println(oneNodeTree)
println("Depth: ${oneNodeTree.getHeight()}")

0

Depth: 0


## Book Solution:

In [9]:
fun <T:Any> BinaryNode<T>.heigh(node: BinaryNode<T>? = this): Int {
    return node?.let { 
        1 + max(heigh(node.leftChild), heigh(node.rightChild))
     } ?: -1
}

println("Deepth T1: ${tree.heigh() }")
println("Deepth T2: ${tree2.heigh() }")
println("Deepth T0: ${oneNodeTree.heigh() }")

Deepth T1: 2
Deepth T2: 4
Deepth T0: 0


## Challenge 2: Serialization of a Binary Tree

In [10]:
import java.lang.IllegalArgumentException

fun addItemSerilized(node: BinaryNode<Int>, data: MutableList<String>) {
    data.add(node.value.toString())
    node.leftChild?.let {
        addItemSerilized(it, data)
    } ?: run { data.add("null") }
    node.rightChild?.let {
        addItemSerilized(it, data)
    } ?: run { data.add("null") }
}

fun serialize(rootNode: BinaryNode<Int>): String {
    val data = mutableListOf<String>()
    addItemSerilized(rootNode, data)
    return data.toString()
}

fun deserialize(data: String): BinaryNode<Int>? {
    if (data.isBlank()) return null
    val cleanDataString = data.substring(1, data.length - 1) 
    val values = cleanDataString.split(",").toMutableList()
    if (values.isEmpty()) return null

    val rootValue = values.removeFirst().toIntOrNull() ?: return null
    val rootNode = BinaryNode(rootValue)
    addChildren(rootNode, values)
    return rootNode
}

fun addChildren(node: BinaryNode<Int>, values: MutableList<String>) {
    if (values.isEmpty()) return

    values.removeFirst().trim().toIntOrNull()?.let { leftValue ->
        val leftNode = BinaryNode(leftValue)
        node.leftChild = leftNode
        addChildren(leftNode, values)
    }
    if (values.isEmpty()) return
    values.removeFirst().trim().toIntOrNull()?.let { rightValue ->
        val rightNode = BinaryNode(rightValue)
        node.rightChild = rightNode
        addChildren(rightNode, values)
    }
}


println(tree)
val treeSeriliazed = serialize(tree) 
println(treeSeriliazed)
val deserializedTree = deserialize(treeSeriliazed)
println(deserializedTree)

 ┌──null
┌──9
| └──8
7
| ┌──5
└──1
 └──0

[7, 1, 0, null, null, 5, null, null, 9, 8, null, null, null]
 ┌──null
┌──9
| └──8
7
| ┌──5
└──1
 └──0


In [11]:
println(tree2)
val tree2Seriliazed = serialize(tree2)
println(tree2Seriliazed)
val deserializedTree2 = deserialize(tree2Seriliazed)
println(deserializedTree2)

 ┌──null
┌──9
| | ┌──null
| └──8
|  | ┌──3
|  └──2
|   └──null
7
| ┌──5
└──1
 └──0

[7, 1, 0, null, null, 5, null, null, 9, 8, 2, null, 3, null, null, null, null]
 ┌──null
┌──9
| | ┌──null
| └──8
|  | ┌──3
|  └──2
|   └──null
7
| ┌──5
└──1
 └──0


## Book Solution

In [12]:
fun <T : Any> BinaryNode<T>.traversePreOrderWithNull(visit: Visitor<T?>) {
    visit(value)
    leftChild?.traversePreOrderWithNull(visit) ?: visit(null)
    rightChild?.traversePreOrderWithNull(visit) ?: visit(null)
}

fun BinaryNode<Int>.serialize(node: BinaryNode<Int> = this): String {
    val list = mutableListOf<Int?>()
    node.traversePreOrderWithNull { list.add(it) }
    return list.toString()
}

fun BinaryNode<Int>.deserializeOptimized(dataSerialized: String): BinaryNode<Int>? {
    if (dataSerialized.isBlank()) return null
    val cleanDataString = dataSerialized.substring(1, dataSerialized.length - 1)
    val list: MutableList<Int?> = cleanDataString.split(",").asReversed().map { it.trim().toIntOrNull() }.toMutableList()
    return deserialize(list)
}

fun BinaryNode<Int>.deserialize(list: MutableList<Int?>): BinaryNode<Int>? {
    val rootValue = list.removeAt(list.size - 1) ?: return null
    val root = BinaryNode<Int>(rootValue)

    root.leftChild = deserialize(list)
    root.rightChild = deserialize(list)
    return root
}

In [13]:
println(tree)
val treeSeriliazed = tree.serialize() 
println(treeSeriliazed)
val deserializedTree = tree.deserializeOptimized(treeSeriliazed)
println(deserializedTree)

 ┌──null
┌──9
| └──8
7
| ┌──5
└──1
 └──0

[7, 1, 0, null, null, 5, null, null, 9, 8, null, null, null]
 ┌──null
┌──9
| └──8
7
| ┌──5
└──1
 └──0


In [14]:
println(tree2)
val tree2Seriliazed = tree2.serialize()
println(tree2Seriliazed)
val deserializedTree2 = tree2.deserializeOptimized(tree2Seriliazed)
println(deserializedTree2)

 ┌──null
┌──9
| | ┌──null
| └──8
|  | ┌──3
|  └──2
|   └──null
7
| ┌──5
└──1
 └──0

[7, 1, 0, null, null, 5, null, null, 9, 8, 2, null, 3, null, null, null, null]
 ┌──null
┌──9
| | ┌──null
| └──8
|  | ┌──3
|  └──2
|   └──null
7
| ┌──5
└──1
 └──0
